In [1]:
import pandas as pd
import numpy as np

# Define sheet name
FILE_NAME = '../Sources/Data.xlsx'

# Get all sheet name in Excel file
sheet_name_list = pd.ExcelFile(FILE_NAME).sheet_names

# Show all sheet name and index number
for index, sheet_name in enumerate(sheet_name_list):
    print(f"{index} : {sheet_name}")

# Read data base on selected SELECT_SHEET_NAME
SELECT_SHEET_NAME = sheet_name_list[5]

# add dataframe for selected sheet
df = pd.read_excel(FILE_NAME, sheet_name= SELECT_SHEET_NAME, skiprows = 5)

0 : Index
1 : Vol. of Txn by Payment System
2 : Value of Txn by Payment System
3 : Vol. of Plastic card usage
4 : Value. of Plastic card usage
5 : number of cards
6 : No of EFTPOS Terminals <2018
7 : No of EFTPOS Terminals 2018+
8 : Payment Cards on ATM & POS 
9 : Mobile,internet banking
10 : Mobile,internet banking add
11 : Bill Payment
12 : Vol. of e-payment
13 : Value. of e-payment
14 : e-Money
15 : Foreign cards & emoney
16 : Thai cards spent abroad
17 : promptpay


In [2]:
### Data Cleansing ###

# Change column name and drop un-related column
df.rename(columns = {"Unnamed: 1" : "Attribute"}, inplace = True)
df.dropna(subset = ["Attribute"], inplace = True)
df.drop(columns = ["Unnamed: 0"], inplace = True)
df.replace("n.a.", np.nan, inplace = True)

# Filter out Total row of column 'Attribute'
df = df[df['Attribute'].str.contains("Total") == False]

# Cleansing Column name (trim and remove leeter after year)
col_list_clean = []
col_list_trim = [col_name.strip() for col_name in df.columns]
for col_name in col_list_trim:
    if len(col_name.split(" ")) <= 2:
        col_list_clean.append(col_name)
    else:
        cleaned_col_name =  " ".join(col_name.split(" ")[:-1])
        col_list_clean.append(cleaned_col_name)
        
# Overwrite existing column with new columns clean column    
df.columns = col_list_clean

In [3]:
### Data Cleansing 2 ###
# Remove /Number from 'Attribute' columns
text_clean_list = []
for text in list(df['Attribute']):
    if text.split(" ")[-1].endswith("/"):
        text_clean_list.append(" ".join(text.split(" ")[:-1]))
    else:
        text_clean_list.append(text)

df['Attribute'] = text_clean_list

In [4]:
### Data Shaping ###

# Unpivot MonthYear Column 
prefix_col = 'Attribute'
level_col_list = [col_name for col_name in df.columns if col_name.startswith(prefix_col)]
value_col_list = [col_name for col_name in df.columns if col_name not in level_col_list]
df = pd.melt(df, id_vars = level_col_list, value_vars = value_col_list)


In [5]:
# rename column
df.rename(columns = {"Attribute" : "ChannelType" ,"variable" : "MonthYear" , "value" : "M_Value"}, inplace = True)

# addcolumn
df['PaymentType'] = 'Plastic cards'

In [6]:
# save output to csv
df.to_csv(f'../Clean/Number_of_cards_Clean.csv', index = False)
